# Load Data

**Download and unzip Dataset**

In [1]:
# download Dataset
!gdown 1h_Xi0ms4kpCvzSsMOsGfhhUqYezXk3s_

# unzip dataset
!unzip "/content/dataset.zip"

Downloading...
From: https://drive.google.com/uc?id=1h_Xi0ms4kpCvzSsMOsGfhhUqYezXk3s_
To: /content/dataset.zip
100% 2.39G/2.39G [00:23<00:00, 103MB/s]
Archive:  /content/dataset.zip
  inflating: dataset/subj_1.mat      
  inflating: dataset/subj_10.mat     
  inflating: dataset/subj_11.mat     
  inflating: dataset/subj_12.mat     
  inflating: dataset/subj_13.mat     
  inflating: dataset/subj_14.mat     
  inflating: dataset/subj_15.mat     
  inflating: dataset/subj_2.mat      
  inflating: dataset/subj_3.mat      
  inflating: dataset/subj_4.mat      
  inflating: dataset/subj_5.mat      
  inflating: dataset/subj_6.mat      
  inflating: dataset/subj_7.mat      
  inflating: dataset/subj_8.mat      
  inflating: dataset/subj_9.mat      


# Install and import

**Install requiered library**

In [2]:
# create requirements.txt 
libraries = [
    'numpy',
    'scipy',
    'gdown',
    'scikit-learn',
    'mne'
]

with open('requirements.txt', 'w') as file:
    for library in libraries:
        file.write(library + '\n')      

In [3]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 8.6 MB/s eta 0:00:00


**Import libraries**

In [4]:
import numpy as np
from scipy.io import loadmat

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split

from mne.decoding import CSP

# Train 1 Model for ALL 15 person data  - Split Leave one out

Train only one model and evaluate model on unseen data

**Preprocess data all data**

In [5]:
# Define the channels of interest
channels_of_interest = [11, 40, 12, 41, 13, 42, 14, 44, 16, 45, 17, 46, 18, 47, 20, 49, 21, 50, 22, 51, 23]

# Initialize empty lists for X and Y data
X = []
Y = []

min_num_trials = float('inf')  # Initialize with a large value

# Loop over each person's data
for i in range(1, 16):
    # Load the .mat file for each person
    data = loadmat(f'dataset/subj_{i}.mat')['data'][0]

    # Extract the classes
    class_1 = data[0][:, channels_of_interest, :]
    class_2 = data[1][:, channels_of_interest, :]
    class_3 = data[2][:, channels_of_interest, :]
    class_4 = data[3][:, channels_of_interest, :]

    # Determine the minimum number of trials among all classes
    min_num_trials = min(min_num_trials, class_1.shape[2], class_2.shape[2], class_3.shape[2], class_4.shape[2])

    # Append the data to X and Y
    X.append(class_1[:, :, :min_num_trials])
    X.append(class_2[:, :, :min_num_trials])
    X.append(class_3[:, :, :min_num_trials])
    X.append(class_4[:, :, :min_num_trials])

    # Create labels for the classes
    num_samples = min_num_trials * 4
    labels = np.zeros((num_samples,))
    labels[min_num_trials:min_num_trials * 2] = 1
    labels[min_num_trials * 2:min_num_trials * 3] = 2
    labels[min_num_trials * 3:] = 3

    # Append the labels to Y
    Y.append(labels)

# Concatenate the data for all people
X = np.concatenate(X, axis=2)
Y = np.concatenate(Y, axis=0)

# Reshape X to match the required shape
X = X.transpose((2, 1, 0))  


**Leave one Out Split**

In [6]:
# Initialize lists for train and test sets
X_train = []
X_test = []
Y_train = []
Y_test = []

# Split the data using leave-one-out method
for i in range(min_num_trials):
    X_train.append(np.concatenate([X[Y != j, :, :] for j in range(4)], axis=0))
    X_test.append(X[Y == i, :, :])
    Y_train.append(np.concatenate([Y[Y != j] for j in range(4)], axis=0))
    Y_test.append(Y[Y == i])

# Convert lists to arrays
X_train = np.concatenate(X_train, axis=0)
X_test = np.concatenate(X_test, axis=0)
Y_train = np.concatenate(Y_train, axis=0)
Y_test = np.concatenate(Y_test, axis=0)

# Create separate datasets for each class
X1_train = X_train[Y_train == 0]
X2_train = X_train[Y_train == 1]
X3_train = X_train[Y_train == 2]
X4_train = X_train[Y_train == 3]

X1_train = X1_train[np.logical_not(np.isnan(np.sum(X1_train, axis=(1, 2))) | np.isinf(np.sum(X1_train, axis=(1, 2))))]
X2_train = X2_train[np.logical_not(np.isnan(np.sum(X2_train, axis=(1, 2))) | np.isinf(np.sum(X2_train, axis=(1, 2))))]
X3_train = X3_train[np.logical_not(np.isnan(np.sum(X3_train, axis=(1, 2))) | np.isinf(np.sum(X3_train, axis=(1, 2))))]
X4_train = np.nan_to_num(X4_train)

X1_test = X_test[Y_test == 0]
X2_test = X_test[Y_test == 1]
X3_test = X_test[Y_test == 2]
X4_test = X_test[Y_test == 3]

X1_test = X1_test[np.logical_not(np.isnan(np.sum(X1_test, axis=(1, 2))) | np.isinf(np.sum(X1_test, axis=(1, 2))))]
X2_test = X2_test[np.logical_not(np.isnan(np.sum(X2_test, axis=(1, 2))) | np.isinf(np.sum(X2_test, axis=(1, 2))))]
X3_test = X3_test[np.logical_not(np.isnan(np.sum(X3_test, axis=(1, 2))) | np.isinf(np.sum(X3_test, axis=(1, 2))))]
X4_test = np.nan_to_num(X4_test)

**Create 3 Classifiers**

In [7]:
# Class 4 vs. Other Classes Classifier
X_classifier1_train = np.concatenate((X1_train, X2_train, X3_train, X4_train), axis=0)
y_classifier1_train = np.concatenate((np.ones(X1_train.shape[0]), np.ones(X2_train.shape[0]),
                                      np.ones(X3_train.shape[0]), np.zeros(X4_train.shape[0])))

X_classifier1_test = np.concatenate((X1_test, X2_test, X3_test, X4_test), axis=0)
y_classifier1_test = np.concatenate((np.ones(X1_test.shape[0]), np.ones(X2_test.shape[0]),
                                    np.ones(X3_test.shape[0]), np.zeros(X4_test.shape[0])))

# Apply CSP to X_classifier1 train and test data
csp1 = CSP(n_components=32, reg=None, log=True)
X_classifier1_train_csp = csp1.fit_transform(X_classifier1_train, y_classifier1_train)
X_classifier1_test_csp = csp1.transform(X_classifier1_test)

# Train LDA on X_classifier1 train data
lda1 = LinearDiscriminantAnalysis()
lda1.fit(X_classifier1_train_csp, y_classifier1_train)

# Predict on train and test data
y_classifier1_train_pred = lda1.predict(X_classifier1_train_csp)
y_classifier1_test_pred = lda1.predict(X_classifier1_test_csp)

# Calculate accuracy for Class 4 vs. Other Classes Classifier
accuracy_train1 = accuracy_score(y_classifier1_train, y_classifier1_train_pred)
accuracy_test1 = accuracy_score(y_classifier1_test, y_classifier1_test_pred)

# Calculate confusion matrices for Class 4 vs. Other Classes Classifier
confusion_matrix_train1 = confusion_matrix(y_classifier1_train, y_classifier1_train_pred)
confusion_matrix_test1 = confusion_matrix(y_classifier1_test, y_classifier1_test_pred)

# Class 3 vs. (Class 1 and 2) Classifier
X_classifier2_train = np.concatenate((X3_train, X1_train, X2_train), axis=0)
y_classifier2_train = np.concatenate((np.zeros(X3_train.shape[0]), np.ones(X1_train.shape[0]),
                                      np.ones(X2_train.shape[0])))

X_classifier2_test = np.concatenate((X3_test, X1_test, X2_test), axis=0)
y_classifier2_test = np.concatenate((np.zeros(X3_test.shape[0]), np.ones(X1_test.shape[0]),
                                    np.ones(X2_test.shape[0])))

# Apply CSP to X_classifier2 train and test data
csp2 = CSP(n_components=32, reg=None, log=True)
X_classifier2_train_csp = csp2.fit_transform(X_classifier2_train, y_classifier2_train)
X_classifier2_test_csp = csp2.transform(X_classifier2_test)

# Train LDA on X_classifier2 train data
lda2 = LinearDiscriminantAnalysis()
lda2.fit(X_classifier2_train_csp, y_classifier2_train)

# Predict on train and test data
y_classifier2_train_pred = lda2.predict(X_classifier2_train_csp)
y_classifier2_test_pred = lda2.predict(X_classifier2_test_csp)

# Calculate accuracy for Class 3 vs. (Class 1 and 2) Classifier
accuracy_train2 = accuracy_score(y_classifier2_train, y_classifier2_train_pred)
accuracy_test2 = accuracy_score(y_classifier2_test, y_classifier2_test_pred)

# Calculate confusion matrices for Class 3 vs. (Class 1 and 2) Classifier
confusion_matrix_train2 = confusion_matrix(y_classifier2_train, y_classifier2_train_pred)
confusion_matrix_test2 = confusion_matrix(y_classifier2_test, y_classifier2_test_pred)

# Class 1 vs. Class 2 Classifier
X_classifier3_train = np.concatenate((X1_train, X2_train), axis=0)
y_classifier3_train = np.concatenate((np.zeros(X1_train.shape[0]), np.ones(X2_train.shape[0])))

X_classifier3_test = np.concatenate((X1_test, X2_test), axis=0)
y_classifier3_test = np.concatenate((np.zeros(X1_test.shape[0]), np.ones(X2_test.shape[0])))

# Apply CSP to X_classifier3 train and test data
csp3 = CSP(n_components=32, reg=None, log=True)
X_classifier3_train_csp = csp3.fit_transform(X_classifier3_train, y_classifier3_train)
X_classifier3_test_csp = csp3.transform(X_classifier3_test)

# Train LDA on X_classifier3 train data
lda3 = LinearDiscriminantAnalysis()
lda3.fit(X_classifier3_train_csp, y_classifier3_train)

# Predict on train and test data
y_classifier3_train_pred = lda3.predict(X_classifier3_train_csp)
y_classifier3_test_pred = lda3.predict(X_classifier3_test_csp)

# Calculate accuracy for Class 1 vs. Class 2 Classifier
accuracy_train3 = accuracy_score(y_classifier3_train, y_classifier3_train_pred)
accuracy_test3 = accuracy_score(y_classifier3_test, y_classifier3_test_pred)

# Calculate confusion matrices for Class 1 vs. Class 2 Classifier
confusion_matrix_train3 = confusion_matrix(y_classifier3_train, y_classifier3_train_pred)
confusion_matrix_test3 = confusion_matrix(y_classifier3_test, y_classifier3_test_pred)

Computing rank from data with rank=None
    Using tolerance 2.7e+02 (2.2e-16 eps * 21 dim * 5.8e+16  max singular value)
    Estimated rank (mag): 21
    MAG: rank 21 computed from 21 data channels with 0 projectors
Reducing data rank from 21 -> 21
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 3.3e+02 (2.2e-16 eps * 21 dim * 7e+16  max singular value)
    Estimated rank (mag): 21
    MAG: rank 21 computed from 21 data channels with 0 projectors
Reducing data rank from 21 -> 21
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 1.7e+02 (2.2e-16 eps * 21 dim * 3.7e+16  max singular value)
    Estimated rank (mag): 21
    MAG: rank 21 computed from 21 data channels with 0 projectors
Reducing data rank from 21 -> 21
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 2.7e+02 (2.2e-16 eps * 21 dim * 5.9e+16  max singular value)
  

**Evaluate on All Test and Train**

In [8]:
# Combine all X train and test data
X_all_train = np.concatenate((X1_train, X2_train, X3_train, X4_train), axis=0)
X_all_test = np.concatenate((X1_test, X2_test, X3_test, X4_test), axis=0)

# Combine all y train and test data
y_all_train = np.concatenate((np.zeros(X1_train.shape[0]), np.ones(X2_train.shape[0]), np.ones(X3_train.shape[0])*2, np.ones(X4_train.shape[0])*3))
y_all_test = np.concatenate((np.zeros(X1_test.shape[0]), np.ones(X2_test.shape[0]), np.ones(X3_test.shape[0])*2, np.ones(X4_test.shape[0])*3))

# Evaluate the cascade of classifiers on the test data
y_pred_all_train = []
y_pred_all_test = []

for x_train in X_all_train:
    x_train_csp1 = csp1.transform(np.array([x_train]))
    x_train_csp2 = csp2.transform(np.array([x_train]))
    x_train_csp3 = csp3.transform(np.array([x_train]))

    if lda1.predict(x_train_csp1) == 1:
        if lda2.predict(x_train_csp2) == 1:
            if lda3.predict(x_train_csp3) == 1:
                y_pred_all_train.append(1)
            else:
                y_pred_all_train.append(0)
        else:
            y_pred_all_train.append(2)
    else:
        y_pred_all_train.append(3)

for x_test in X_all_test:
    x_test_csp1 = csp1.transform(np.array([x_test]))
    x_test_csp2 = csp2.transform(np.array([x_test]))
    x_test_csp3 = csp3.transform(np.array([x_test]))

    if lda1.predict(x_test_csp1) == 1:
        if lda2.predict(x_test_csp2) == 1:
            if lda3.predict(x_test_csp3) == 1:
                y_pred_all_test.append(1)
            else:
                y_pred_all_test.append(0)
        else:
            y_pred_all_test.append(2)
    else:
        y_pred_all_test.append(3)

**Print Results**

In [9]:
# Print confusion matrix and accuracy for Class 4 vs. Other Classes Classifier
print("Confusion Matrix for Class 4 vs. Other Classes Classifier (Train):")
print(confusion_matrix_train1)
print("Accuracy for Class 4 vs. Other Classes Classifier (Train):", accuracy_train1)
print("\nConfusion Matrix for Class 4 vs. Other Classes Classifier (Test):")
print(confusion_matrix_test1)
print("Accuracy for Class 4 vs. Other Classes Classifier (Test):", accuracy_test1)
print("--------------------------------------------------")

# Print confusion matrix and accuracy for Class 3 vs. (Class 1 and 2) Classifier
print("Confusion Matrix for Class 3 vs. (Class 1 and 2) Classifier (Train):")
print(confusion_matrix_train2)
print("Accuracy for Class 3 vs. (Class 1 and 2) Classifier (Train):", accuracy_train2)
print("\nConfusion Matrix for Class 3 vs. (Class 1 and 2) Classifier (Test):")
print(confusion_matrix_test2)
print("Accuracy for Class 3 vs. (Class 1 and 2) Classifier (Test):", accuracy_test2)
print("--------------------------------------------------")

# Print confusion matrix and accuracy for Class 1 vs. Class 2 Classifier
print("Confusion Matrix for Class 1 vs. Class 2 Classifier (Train):")
print(confusion_matrix_train3)
print("Accuracy for Class 1 vs. Class 2 Classifier (Train):", accuracy_train3)
print("\nConfusion Matrix for Class 1 vs. Class 2 Classifier (Test):")
print(confusion_matrix_test3)
print("Accuracy for Class 1 vs. Class 2 Classifier (Test):", accuracy_test3)
print("--------------------------------------------------")

# Print accuracy for classifier 2 and 3
print("Accuracy for Class 3 vs. (Class 1 and 2) Classifier (Train):", accuracy_train2)
print("Accuracy for Class 1 vs. Class 2 Classifier (Train):", accuracy_train3)

print("--------------------------------------------------")

# Print confusion matrix and accuracy for the cascade of classifiers
print("Confusion Matrix for the Cascade of Classifiers (Train):")
print(confusion_matrix(y_all_train, y_pred_all_train))
print("Accuracy for the Cascade of Classifiers (Train):", accuracy_score(y_all_train, y_pred_all_train))
print("\nConfusion Matrix for the Cascade of Classifiers (Test):")
print(confusion_matrix(y_all_test, y_pred_all_test))
print("Accuracy for the Cascade of Classifiers (Test):", accuracy_score(y_all_test, y_pred_all_test))


Confusion Matrix for Class 4 vs. Other Classes Classifier (Train):
[[ 4176  8064]
 [ 1728 34992]]
Accuracy for Class 4 vs. Other Classes Classifier (Train): 0.8

Confusion Matrix for Class 4 vs. Other Classes Classifier (Test):
[[ 87 168]
 [ 36 729]]
Accuracy for Class 4 vs. Other Classes Classifier (Test): 0.8
--------------------------------------------------
Confusion Matrix for Class 3 vs. (Class 1 and 2) Classifier (Train):
[[ 4464  7776]
 [ 2448 22032]]
Accuracy for Class 3 vs. (Class 1 and 2) Classifier (Train): 0.7215686274509804

Confusion Matrix for Class 3 vs. (Class 1 and 2) Classifier (Test):
[[ 93 162]
 [ 51 459]]
Accuracy for Class 3 vs. (Class 1 and 2) Classifier (Test): 0.7215686274509804
--------------------------------------------------
Confusion Matrix for Class 1 vs. Class 2 Classifier (Train):
[[8544 3696]
 [4224 8016]]
Accuracy for Class 1 vs. Class 2 Classifier (Train): 0.6764705882352942

Confusion Matrix for Class 1 vs. Class 2 Classifier (Test):
[[178  77]
 [

In [10]:
# Print W(CSP) and W(LDA):

# Retrieve the weights for CSP
csp1_weights = csp1.filters_
csp2_weights = csp1.filters_
csp3_weights = csp1.filters_

# Retrieve the weights for LDA
lda1_weights = lda1.coef_
lda2_weights = lda1.coef_
lda3_weights = lda1.coef_

# Print the weights for CSP
print("CSP weights:")
print(csp1_weights)
print("----")
print(csp2_weights)
print("----")
print(csp3_weights)
print("----")

# Print the weights for LDA
print("LDA weights:")
print(lda1_weights)
print("----")
print(lda2_weights)
print("----")
print(lda3_weights)
print("----")

CSP weights:
[[-1.66183658e+01 -5.89211187e+00  5.83679083e+01 -1.43195597e+01
  -8.29519263e+01  3.28855924e+01  5.97667150e+01 -2.51200090e+01
  -4.65715597e+01 -3.29341973e+01  4.17557599e+01  5.39644241e+01
  -1.71743784e+01  1.38427959e+02 -2.20213326e+01 -6.14022125e+02
   3.91632108e+01  6.87207714e+02  4.39732163e-02 -2.20228902e+02
  -1.37292774e+01]
 [-2.37900324e+00 -4.61097337e+01  3.90713188e+00  2.84551397e+02
  -2.43555886e+01 -3.73167681e+02  1.96971867e+01  3.73048782e+02
   8.32339003e+01 -5.49895505e+02 -7.31205529e+01  5.78532430e+02
  -1.10761336e+01 -3.26700482e+02 -1.04834681e+02  1.02914589e+02
   1.14788071e+01 -4.12959541e+01  2.74633186e+02  3.20890661e-01
  -1.79443751e+02]
 [-4.12830597e+02 -2.03679932e+02  1.24690092e+03  1.36665960e+02
  -1.23136371e+03  2.18225455e+02  3.15013941e+02 -5.19752091e+02
   3.48450844e+02  1.60677818e+02 -1.34330410e+02  8.69867091e+02
  -4.46842090e+02 -7.87826182e+02  1.09887695e+03  7.94688016e+01
  -1.21664190e+03  3.6614

# Train 1 Model for ALL 15 person data  - Split Randomly

Train only one model and evaluate model on unseen data ( different to last is that we dont use LOO method for split )

In [11]:
# Define the channels of interest
channels_of_interest = [11, 40, 12, 41, 13, 42, 14, 44, 16, 45, 17, 46, 18, 47, 20, 49, 21, 50, 22, 51, 23]
# Initialize empty lists for X and Y data
X = []
Y = []
min_num_trials = float('inf')  # Initialize with a large value
# Loop over each person's data
for i in range(1, 16):
    # Load the .mat file for each person
    data = loadmat(f'dataset/subj_{i}.mat')['data'][0]
    # Extract the classes
    class_1 = data[0][:, channels_of_interest, :]
    class_2 = data[1][:, channels_of_interest, :]
    class_3 = data[2][:, channels_of_interest, :]
    class_4 = data[3][:, channels_of_interest, :]
    # Determine the minimum number of trials among all classes
    min_num_trials = min(min_num_trials, class_1.shape[2], class_2.shape[2], class_3.shape[2], class_4.shape[2])
    # Append the data to X and Y
    X.append(class_1[:, :, :min_num_trials])
    X.append(class_2[:, :, :min_num_trials])
    X.append(class_3[:, :, :min_num_trials])
    X.append(class_4[:, :, :min_num_trials])
    # Create labels for the classes
    num_samples = min_num_trials * 4
    labels = np.zeros((num_samples,))
    labels[min_num_trials:min_num_trials * 2] = 1
    labels[min_num_trials * 2:min_num_trials * 3] = 2
    labels[min_num_trials * 3:] = 3
    # Append the labels to Y
    Y.append(labels)

In [12]:
X = np.concatenate(X, axis=2)
Y = np.concatenate(Y, axis=0)
X = X.transpose((2, 1, 0))
X1 = X[Y == 0]
X2 = X[Y == 1]
X3 = X[Y == 2]
X4 = X[Y == 3]
# Delete samples with NaN and inf values 
X1 = X1[np.logical_not(np.isnan(np.sum(X1, axis=(1, 2))) | np.isinf(np.sum(X1, axis=(1, 2))))]
X2 = X2[np.logical_not(np.isnan(np.sum(X2, axis=(1, 2))) | np.isinf(np.sum(X2, axis=(1, 2))))]
X3 = X3[np.logical_not(np.isnan(np.sum(X3, axis=(1, 2))) | np.isinf(np.sum(X3, axis=(1, 2))))]
X4 = np.nan_to_num(X4) 

In [13]:
# Class 4 vs. Other Classes Classifier
X_classifier1 = np.concatenate((X1, X2, X3, X4), axis=0)
y_classifier1 = np.concatenate((np.ones(X1.shape[0]), np.ones(X2.shape[0]), np.ones(X3.shape[0]) , np.zeros(X4.shape[0])))

X_classifier1_train, X_classifier1_test, y_classifier1_train, y_classifier1_test = train_test_split(
    X_classifier1, y_classifier1, test_size=0.2, random_state=42)

# Class 3 vs. (Class 1 and 2) Classifier
X_classifier2 = np.concatenate((X3, X1, X2), axis=0)
y_classifier2 = np.concatenate((np.zeros(X3.shape[0]), np.ones(X1.shape[0]), np.ones(X2.shape[0])))

X_classifier2_train, X_classifier2_test, y_classifier2_train, y_classifier2_test = train_test_split(
    X_classifier2, y_classifier2, test_size=0.2, random_state=42)

# Class 1 vs. Class 2 Classifier
X_classifier3 = np.concatenate((X1, X2), axis=0)
X_classifier3_train, X_classifier3_test, y_classifier3_train, y_classifier3_test = train_test_split(
    np.concatenate((X1, X2), axis=0), np.concatenate((np.zeros(X1.shape[0]), np.ones(X2.shape[0]))),
    test_size=0.2, random_state=42)

# Apply CSP to X_classifier1 train and test data
csp1 = CSP(n_components=32, reg=None, log=True)
X_classifier1_train_csp = csp1.fit_transform(X_classifier1_train, y_classifier1_train)
X_classifier1_test_csp = csp1.transform(X_classifier1_test)

# Apply CSP to X_classifier2 train and test data
csp2 = CSP(n_components=32, reg=None, log=True)
X_classifier2_train_csp = csp2.fit_transform(X_classifier2_train, y_classifier2_train)
X_classifier2_test_csp = csp2.transform(X_classifier2_test)

# Apply CSP to X_classifier3 train and test data
csp3 = CSP(n_components=32, reg=None, log=True)
X_classifier3_train_csp = csp3.fit_transform(X_classifier3_train, y_classifier3_train)
X_classifier3_test_csp = csp3.transform(X_classifier3_test)

Computing rank from data with rank=None
    Using tolerance 37 (2.2e-16 eps * 21 dim * 7.9e+15  max singular value)
    Estimated rank (mag): 21
    MAG: rank 21 computed from 21 data channels with 0 projectors
Reducing data rank from 21 -> 21
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 43 (2.2e-16 eps * 21 dim * 9.3e+15  max singular value)
    Estimated rank (mag): 21
    MAG: rank 21 computed from 21 data channels with 0 projectors
Reducing data rank from 21 -> 21
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 23 (2.2e-16 eps * 21 dim * 4.9e+15  max singular value)
    Estimated rank (mag): 21
    MAG: rank 21 computed from 21 data channels with 0 projectors
Reducing data rank from 21 -> 21
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 36 (2.2e-16 eps * 21 dim * 7.7e+15  max singular value)
    Estimated rank (

In [14]:
# Step 1: Class 4 vs. Other Classes Classifier
lda1 = LinearDiscriminantAnalysis()
lda1.fit(X_classifier1_train_csp, y_classifier1_train)

# Predict on train and test data
y_classifier1_train_pred = lda1.predict(X_classifier1_train_csp)
y_classifier1_test_pred = lda1.predict(X_classifier1_test_csp)

# Calculate accuracy
accuracy_train1 = accuracy_score(y_classifier1_train, y_classifier1_train_pred)
accuracy_test1 = accuracy_score(y_classifier1_test, y_classifier1_test_pred)

# Calculate confusion matrices
confusion_matrix_train = confusion_matrix(y_classifier1_train, y_classifier1_train_pred)
confusion_matrix_test = confusion_matrix(y_classifier1_test, y_classifier1_test_pred)

print("Classifier 1 - Class 4 vs. Other Classes")
print("Train Accuracy:", accuracy_train1)
print("Test Accuracy:", accuracy_test1)
print("Confusion Matrix Train :")
print(confusion_matrix_train)
print("Confusion Matrix Test :")
print(confusion_matrix_test)
print()

Classifier 1 - Class 4 vs. Other Classes
Train Accuracy: 0.8125
Test Accuracy: 0.7647058823529411
Confusion Matrix Train :
[[ 80 127]
 [ 26 583]]
Confusion Matrix Test :
[[  9  39]
 [  9 147]]



In [15]:
# Step 2: Class 3 vs. (Class 1 and 2) Classifier
lda2 = LinearDiscriminantAnalysis()
lda2.fit(X_classifier2_train_csp, y_classifier2_train)

# Predict on train and test data
y_classifier2_train_pred = lda2.predict(X_classifier2_train_csp)
y_classifier2_test_pred = lda2.predict(X_classifier2_test_csp)

# Calculate accuracy
accuracy_train2 = accuracy_score(y_classifier2_train, y_classifier2_train_pred)
accuracy_test2 = accuracy_score(y_classifier2_test, y_classifier2_test_pred)

# Calculate confusion matrices
confusion_matrix_train = confusion_matrix(y_classifier2_train, y_classifier2_train_pred)
confusion_matrix_test = confusion_matrix(y_classifier2_test, y_classifier2_test_pred)

print("Classifier 2 - Class 3 vs. (Class 1 and 2)")
print("Train Accuracy:", accuracy_train2)
print("Test Accuracy:", accuracy_test2)
print("Confusion Matrix Train :")
print(confusion_matrix_train)
print("Confusion Matrix Test :")
print(confusion_matrix_test)

Classifier 2 - Class 3 vs. (Class 1 and 2)
Train Accuracy: 0.7434640522875817
Test Accuracy: 0.5816993464052288
Confusion Matrix Train :
[[ 90 112]
 [ 45 365]]
Confusion Matrix Test :
[[ 6 47]
 [17 83]]


In [16]:
# Step 3: Class 1 vs. Class 2 Classifier
lda3 = LinearDiscriminantAnalysis()
lda3.fit(X_classifier3_train_csp, y_classifier3_train)

# Predict on train and test data
y_classifier3_train_pred = lda3.predict(X_classifier3_train_csp)
y_classifier3_test_pred = lda3.predict(X_classifier3_test_csp)

# Calculate accuracy
accuracy_train3 = accuracy_score(y_classifier3_train, y_classifier3_train_pred)
accuracy_test3 = accuracy_score(y_classifier3_test, y_classifier3_test_pred)

# Calculate confusion matrix
confusion_matrix_train = confusion_matrix(y_classifier3_train, y_classifier3_train_pred)
confusion_matrix_test = confusion_matrix(y_classifier3_test, y_classifier3_test_pred)

print("Classifier 3 - Class 1 vs. Class 2")
print("Train Accuracy:", accuracy_train3)
print("Test Accuracy:", accuracy_test3)
print("Confusion Matrix Train :")
print(confusion_matrix_train)
print("Confusion Matrix Test :")
print(confusion_matrix_test)

Classifier 3 - Class 1 vs. Class 2
Train Accuracy: 0.7034313725490197
Test Accuracy: 0.6176470588235294
Confusion Matrix Train :
[[145  57]
 [ 64 142]]
Confusion Matrix Test :
[[35 18]
 [21 28]]


# Train 15 Models for each Person ( Split LOO)

Train 15 model for 15 person ( using CSP and LDA) and print acc and confiussion Matrix

In [17]:
# Define the channels of interest
channels_of_interest = [11, 40, 12, 41, 13, 42, 14, 44, 16, 45, 17, 46, 18, 47, 20, 49, 21, 50, 22, 51, 23]

# Loop over each person
for i in range(1, 16):
  print("-------------------------------------------------------------------")
  print("Person", i)
  print("-------------------------------------------------------------------")

  ## load data 

  # Initialize empty lists for X and Y data
  X = []
  Y = []

  # Load the .mat file for each person
  data = loadmat(f'dataset/subj_{i}.mat')['data'][0]
  min_num_trials = float('inf')  # Initialize with a large value

  # Extract the classes
  class_1 = data[0][:, channels_of_interest, :]
  class_2 = data[1][:, channels_of_interest, :]
  class_3 = data[2][:, channels_of_interest, :]
  class_4 = data[3][:, channels_of_interest, :]

  # Determine the minimum number of trials among all classes
  min_num_trials = min(min_num_trials, class_1.shape[2], class_2.shape[2], class_3.shape[2], class_4.shape[2])

  # Append the data to X and Y
  X.append(class_1[:, :, :min_num_trials])
  X.append(class_2[:, :, :min_num_trials])
  X.append(class_3[:, :, :min_num_trials])
  X.append(class_4[:, :, :min_num_trials])

  # Create labels for the classes
  num_samples = min_num_trials * 4
  labels = np.zeros((num_samples,))
  labels[min_num_trials:min_num_trials * 2] = 1
  labels[min_num_trials * 2:min_num_trials * 3] = 2
  labels[min_num_trials * 3:] = 3

  # Append the labels to Y
  Y.append(labels)

  X = np.concatenate(X, axis=2)
  Y = np.concatenate(Y, axis=0)
  X = X.transpose((2, 1, 0))

  ## Leave one Out Split

  # Initialize lists for train and test sets
  X_train = []
  X_test = []
  Y_train = []
  Y_test = []

  # Split the data using leave-one-out method
  for i in range(min_num_trials):
      X_train.append(np.concatenate([X[Y != j, :, :] for j in range(4)], axis=0))
      X_test.append(X[Y == i, :, :])
      Y_train.append(np.concatenate([Y[Y != j] for j in range(4)], axis=0))
      Y_test.append(Y[Y == i])

  # Convert lists to arrays
  X_train = np.concatenate(X_train, axis=0)
  X_test = np.concatenate(X_test, axis=0)
  Y_train = np.concatenate(Y_train, axis=0)
  Y_test = np.concatenate(Y_test, axis=0)

  # Create separate datasets for each class
  X1_train = X_train[Y_train == 0]
  X2_train = X_train[Y_train == 1]
  X3_train = X_train[Y_train == 2]
  X4_train = X_train[Y_train == 3]

  X1_train = X1_train[np.logical_not(np.isnan(np.sum(X1_train, axis=(1, 2))) | np.isinf(np.sum(X1_train, axis=(1, 2))))]
  X2_train = X2_train[np.logical_not(np.isnan(np.sum(X2_train, axis=(1, 2))) | np.isinf(np.sum(X2_train, axis=(1, 2))))]
  X3_train = X3_train[np.logical_not(np.isnan(np.sum(X3_train, axis=(1, 2))) | np.isinf(np.sum(X3_train, axis=(1, 2))))]
  X4_train = np.nan_to_num(X4_train)

  X1_test = X_test[Y_test == 0]
  X2_test = X_test[Y_test == 1]
  X3_test = X_test[Y_test == 2]
  X4_test = X_test[Y_test == 3]

  X1_test = X1_test[np.logical_not(np.isnan(np.sum(X1_test, axis=(1, 2))) | np.isinf(np.sum(X1_test, axis=(1, 2))))]
  X2_test = X2_test[np.logical_not(np.isnan(np.sum(X2_test, axis=(1, 2))) | np.isinf(np.sum(X2_test, axis=(1, 2))))]
  X3_test = X3_test[np.logical_not(np.isnan(np.sum(X3_test, axis=(1, 2))) | np.isinf(np.sum(X3_test, axis=(1, 2))))]
  X4_test = np.nan_to_num(X4_test)

  ## Create 3 Classifiers

  # Class 4 vs. Other Classes Classifier
  X_classifier1_train = np.concatenate((X1_train, X2_train, X3_train, X4_train), axis=0)
  y_classifier1_train = np.concatenate((np.ones(X1_train.shape[0]), np.ones(X2_train.shape[0]),
                                        np.ones(X3_train.shape[0]), np.zeros(X4_train.shape[0])))

  X_classifier1_test = np.concatenate((X1_test, X2_test, X3_test, X4_test), axis=0)
  y_classifier1_test = np.concatenate((np.ones(X1_test.shape[0]), np.ones(X2_test.shape[0]),
                                      np.ones(X3_test.shape[0]), np.zeros(X4_test.shape[0])))

  # Apply CSP to X_classifier1 train and test data
  csp1 = CSP(n_components=32, reg=None, log=True)
  X_classifier1_train_csp = csp1.fit_transform(X_classifier1_train, y_classifier1_train)
  X_classifier1_test_csp = csp1.transform(X_classifier1_test)

  # Train LDA on X_classifier1 train data
  lda1 = LinearDiscriminantAnalysis()
  lda1.fit(X_classifier1_train_csp, y_classifier1_train)

  # Predict on train and test data
  y_classifier1_train_pred = lda1.predict(X_classifier1_train_csp)
  y_classifier1_test_pred = lda1.predict(X_classifier1_test_csp)

  # Calculate accuracy for Class 4 vs. Other Classes Classifier
  accuracy_train1 = accuracy_score(y_classifier1_train, y_classifier1_train_pred)
  accuracy_test1 = accuracy_score(y_classifier1_test, y_classifier1_test_pred)

  # Calculate confusion matrices for Class 4 vs. Other Classes Classifier
  confusion_matrix_train1 = confusion_matrix(y_classifier1_train, y_classifier1_train_pred)
  confusion_matrix_test1 = confusion_matrix(y_classifier1_test, y_classifier1_test_pred)

  # Class 3 vs. (Class 1 and 2) Classifier
  X_classifier2_train = np.concatenate((X3_train, X1_train, X2_train), axis=0)
  y_classifier2_train = np.concatenate((np.zeros(X3_train.shape[0]), np.ones(X1_train.shape[0]),
                                        np.ones(X2_train.shape[0])))

  X_classifier2_test = np.concatenate((X3_test, X1_test, X2_test), axis=0)
  y_classifier2_test = np.concatenate((np.zeros(X3_test.shape[0]), np.ones(X1_test.shape[0]),
                                      np.ones(X2_test.shape[0])))

  # Apply CSP to X_classifier2 train and test data
  csp2 = CSP(n_components=32, reg=None, log=True)
  X_classifier2_train_csp = csp2.fit_transform(X_classifier2_train, y_classifier2_train)
  X_classifier2_test_csp = csp2.transform(X_classifier2_test)

  # Train LDA on X_classifier2 train data
  lda2 = LinearDiscriminantAnalysis()
  lda2.fit(X_classifier2_train_csp, y_classifier2_train)

  # Predict on train and test data
  y_classifier2_train_pred = lda2.predict(X_classifier2_train_csp)
  y_classifier2_test_pred = lda2.predict(X_classifier2_test_csp)

  # Calculate accuracy for Class 3 vs. (Class 1 and 2) Classifier
  accuracy_train2 = accuracy_score(y_classifier2_train, y_classifier2_train_pred)
  accuracy_test2 = accuracy_score(y_classifier2_test, y_classifier2_test_pred)

  # Calculate confusion matrices for Class 3 vs. (Class 1 and 2) Classifier
  confusion_matrix_train2 = confusion_matrix(y_classifier2_train, y_classifier2_train_pred)
  confusion_matrix_test2 = confusion_matrix(y_classifier2_test, y_classifier2_test_pred)

  # Class 1 vs. Class 2 Classifier
  X_classifier3_train = np.concatenate((X1_train, X2_train), axis=0)
  y_classifier3_train = np.concatenate((np.zeros(X1_train.shape[0]), np.ones(X2_train.shape[0])))

  X_classifier3_test = np.concatenate((X1_test, X2_test), axis=0)
  y_classifier3_test = np.concatenate((np.zeros(X1_test.shape[0]), np.ones(X2_test.shape[0])))

  # Apply CSP to X_classifier3 train and test data
  csp3 = CSP(n_components=32, reg=None, log=True)
  X_classifier3_train_csp = csp3.fit_transform(X_classifier3_train, y_classifier3_train)
  X_classifier3_test_csp = csp3.transform(X_classifier3_test)

  # Train LDA on X_classifier3 train data
  lda3 = LinearDiscriminantAnalysis()
  lda3.fit(X_classifier3_train_csp, y_classifier3_train)

  # Predict on train and test data
  y_classifier3_train_pred = lda3.predict(X_classifier3_train_csp)
  y_classifier3_test_pred = lda3.predict(X_classifier3_test_csp)

  # Calculate accuracy for Class 1 vs. Class 2 Classifier
  accuracy_train3 = accuracy_score(y_classifier3_train, y_classifier3_train_pred)
  accuracy_test3 = accuracy_score(y_classifier3_test, y_classifier3_test_pred)

  # Calculate confusion matrices for Class 1 vs. Class 2 Classifier
  confusion_matrix_train3 = confusion_matrix(y_classifier3_train, y_classifier3_train_pred)
  confusion_matrix_test3 = confusion_matrix(y_classifier3_test, y_classifier3_test_pred)

  # Combine all X train and test data
  X_all_train = np.concatenate((X1_train, X2_train, X3_train, X4_train), axis=0)
  X_all_test = np.concatenate((X1_test, X2_test, X3_test, X4_test), axis=0)

  # Combine all y train and test data
  y_all_train = np.concatenate((np.zeros(X1_train.shape[0]), np.ones(X2_train.shape[0]), np.ones(X3_train.shape[0])*2, np.ones(X4_train.shape[0])*3))
  y_all_test = np.concatenate((np.zeros(X1_test.shape[0]), np.ones(X2_test.shape[0]), np.ones(X3_test.shape[0])*2, np.ones(X4_test.shape[0])*3))

  ## Evaluate on All Test and Train

  # Evaluate the cascade of classifiers on the test data
  y_pred_all_train = []
  y_pred_all_test = []

  for x_train in X_all_train:
      x_train_csp1 = csp1.transform(np.array([x_train]))
      x_train_csp2 = csp2.transform(np.array([x_train]))
      x_train_csp3 = csp3.transform(np.array([x_train]))

      if lda1.predict(x_train_csp1) == 1:
          if lda2.predict(x_train_csp2) == 1:
              if lda3.predict(x_train_csp3) == 1:
                  y_pred_all_train.append(1)
              else:
                  y_pred_all_train.append(0)
          else:
              y_pred_all_train.append(2)
      else:
          y_pred_all_train.append(3)

  for x_test in X_all_test:
      x_test_csp1 = csp1.transform(np.array([x_test]))
      x_test_csp2 = csp2.transform(np.array([x_test]))
      x_test_csp3 = csp3.transform(np.array([x_test]))

      if lda1.predict(x_test_csp1) == 1:
          if lda2.predict(x_test_csp2) == 1:
              if lda3.predict(x_test_csp3) == 1:
                  y_pred_all_test.append(1)
              else:
                  y_pred_all_test.append(0)
          else:
              y_pred_all_test.append(2)
      else:
          y_pred_all_test.append(3)

  ## Print results

  # Print confusion matrix and accuracy for Class 4 vs. Other Classes Classifier
  print("Confusion Matrix for Class 4 vs. Other Classes Classifier (Train):")
  print(confusion_matrix_train1)
  print("Accuracy for Class 4 vs. Other Classes Classifier (Train):", accuracy_train1)
  print("\nConfusion Matrix for Class 4 vs. Other Classes Classifier (Test):")
  print(confusion_matrix_test1)
  print("Accuracy for Class 4 vs. Other Classes Classifier (Test):", accuracy_test1)
  print("--------------------------------------------------")

  # Print confusion matrix and accuracy for Class 3 vs. (Class 1 and 2) Classifier
  print("Confusion Matrix for Class 3 vs. (Class 1 and 2) Classifier (Train):")
  print(confusion_matrix_train2)
  print("Accuracy for Class 3 vs. (Class 1 and 2) Classifier (Train):", accuracy_train2)
  print("\nConfusion Matrix for Class 3 vs. (Class 1 and 2) Classifier (Test):")
  print(confusion_matrix_test2)
  print("Accuracy for Class 3 vs. (Class 1 and 2) Classifier (Test):", accuracy_test2)
  print("--------------------------------------------------")

  # Print confusion matrix and accuracy for Class 1 vs. Class 2 Classifier
  print("Confusion Matrix for Class 1 vs. Class 2 Classifier (Train):")
  print(confusion_matrix_train3)
  print("Accuracy for Class 1 vs. Class 2 Classifier (Train):", accuracy_train3)
  print("\nConfusion Matrix for Class 1 vs. Class 2 Classifier (Test):")
  print(confusion_matrix_test3)
  print("Accuracy for Class 1 vs. Class 2 Classifier (Test):", accuracy_test3)
  print("--------------------------------------------------")

  # Print accuracy for classifier 2 and 3
  print("Accuracy for Class 3 vs. (Class 1 and 2) Classifier (Train):", accuracy_train2)
  print("Accuracy for Class 1 vs. Class 2 Classifier (Train):", accuracy_train3)

  print("--------------------------------------------------")

  # Print confusion matrix and accuracy for the cascade of classifiers
  print("Confusion Matrix for the Cascade of Classifiers (Train):")
  print(confusion_matrix(y_all_train, y_pred_all_train))
  print("Accuracy for the Cascade of Classifiers (Train):", accuracy_score(y_all_train, y_pred_all_train))
  print("\nConfusion Matrix for the Cascade of Classifiers (Test):")
  print(confusion_matrix(y_all_test, y_pred_all_test))
  print("Accuracy for the Cascade of Classifiers (Test):", accuracy_score(y_all_test, y_pred_all_test))

  # Print W(CSP) and W(LDA):

  # # Retrieve the weights for CSP
  # csp1_weights = csp1.filters_
  # csp2_weights = csp1.filters_
  # csp3_weights = csp1.filters_

  # # Retrieve the weights for LDA
  # lda1_weights = lda1.coef_
  # lda2_weights = lda1.coef_
  # lda3_weights = lda1.coef_

  # # Print the weights for CSP
  # print("CSP weights:")
  # print(csp1_weights)
  # print("----")
  # print(csp2_weights)
  # print("----")
  # print(csp3_weights)
  # print("----")

  # # Print the weights for LDA
  # print("LDA weights:")
  # print(lda1_weights)
  # print("----")
  # print(lda2_weights)
  # print("----")
  # print(lda3_weights)
  # print("----")


-------------------------------------------------------------------
Person 1
-------------------------------------------------------------------
Computing rank from data with rank=None
    Using tolerance 49 (2.2e-16 eps * 21 dim * 1e+16  max singular value)
    Estimated rank (mag): 21
    MAG: rank 21 computed from 21 data channels with 0 projectors
Reducing data rank from 21 -> 21
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 81 (2.2e-16 eps * 21 dim * 1.7e+16  max singular value)
    Estimated rank (mag): 21
    MAG: rank 21 computed from 21 data channels with 0 projectors
Reducing data rank from 21 -> 21
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 45 (2.2e-16 eps * 21 dim * 9.8e+15  max singular value)
    Estimated rank (mag): 21
    MAG: rank 21 computed from 21 data channels with 0 projectors
Reducing data rank from 21 -> 21
Estimating covariance using EMPIRICAL